# **VIDEO GENERATION AGENT**

In [22]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os
from pydantic import BaseModel
import json
from datetime import datetime
from pydantic import BaseModel, Field
from typing import List, Optional

load_dotenv()
if os.environ.get('GEMINI_API_KEY'):
    print("The key loaded successfully")
else:
    raise ValueError("ERROR LOADING THE KEY")

The key loaded successfully


# **GENERATING TRENDING TOPICS**

In [23]:
def generate_trending_topic():
    prompt = "As a senior market trend analyst and social media manager for the creators you have extreme knowledge about the topics or trends following today please output the only idea in text format no explanation or anything other not needed."
    client = ChatGoogleGenerativeAI(model = "gemini-2.5-flash",temperature=0)
    response = client.invoke(prompt)
    return response.content

# **PROMPTS**

In [24]:
SYSTEM_PROMPT = """
You are an expert AI Cinematographer and Sound Designer. Your role is to take a creative strategy and translate it into precise technical instructions for Google Veo 3.1.

Instructions for Scene Generation:
Camera Movement: Always specify a camera motion (e.g., 'dolly-in', 'slow orbit', 'low-angle tilt').
Cinematic Lighting: Use professional lighting terms (e.g., 'volumetric fog', 'rim lighting', 'soft-box diffusion').
Soundscape Logic: Describe sounds that match the visual actions for Veo’s native audio engine.

Output (Strict JSON):
{{
  "veo_prompt": "A detailed, 1-paragraph cinematic description of the scene including camera movement and lighting.",
  "motion_bucket_id": 85,
  "audio_description": "A specific description of the ambient and foley sounds.",
  "video_metadata": {{
    "resolution": "1080p",
    "aspect_ratio": "9:16",
    "fps": 24
  }}
}}
"""

USER_PROMPT = generate_trending_topic()

ChatGoogleGenerativeAIError: Error calling model 'gemini-2.5-flash' (RESOURCE_EXHAUSTED): 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 31.282523497s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeT

# **PYDANTIC CLASSES AND FUNCTION**

In [ ]:


class VideoExecutorStrategy(BaseModel):
    video_prompt: str = Field(
        ..., 
        description="Detailed cinematic description for Veo 3.1 including camera movement and lighting."
    )
    motion_bucket_id: int = Field(
        default=85, 
        ge=1, le=255, 
        description="Motion intensity: 1 for static, 255 for extreme movement."
    )
    audio_prompt: str = Field(
        ..., 
        description="Description of the native audio soundscape (ambient sounds, music, foley)."
    )
    overlay_text: str = Field(
        ..., 
        description="The hook/headline text to be displayed on the video."
    )
    aspect_ratio: str = Field(
        default="9:16", 
        description="The frame dimensions (e.g., 9:16 for vertical, 16:9 for wide)."
    )
    video_title: str = Field(..., description="Optimized title for the video post.")
    tags: List[str] = Field(default_factory=list, description="List of 5 relevant SEO hashtags.")

    class Config:
        schema_extra = {
            "example": {
                "video_prompt": "A slow-motion tracking shot of a neon-lit futuristic garden...",
                "motion_bucket_id": 120,
                "audio_prompt": "Soft electronic hum with the sound of wind chimes.",
                "overlay_text": "Gardening in 2077",
                "aspect_ratio": "9:16",
                "video_title": "Future Garden Aesthetics",
                "tags": ["#future", "#aiart", "#gardening"]
            }
        }


def log_to_daily_jsonl(data_object, folder="logs"):
    if not os.path.exists(folder):
        os.makedirs(folder)
    date_str = datetime.now().strftime("%Y-%m-%d")
    filename = os.path.join(folder, f"{date_str}_pinterest.jsonl")
    if hasattr(data_object, 'model_dump'):
        entry = data_object.model_dump()
    else:
        entry = data_object.dict()
    entry["logged_at"] = datetime.now().isoformat()
    with open(filename, "a", encoding="utf-8") as f:
        f.write(json.dumps(entry) + "\n")
    return filename

C:\Users\souga\AppData\Local\Temp\ipykernel_23308\1280812415.py:4: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class VideoExecutorStrategy(BaseModel):


# **OUTPUT**

In [ ]:
dynamic_prompt = ChatPromptTemplate.from_messages([
    ('system',SYSTEM_PROMPT),
    ('human','topic')
])

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0)
structured_model = model.with_structured_output(VideoExecutorStrategy)
chain = dynamic_prompt | structured_model

try:
    print("Generating Strategy JSON...")
    result = chain.invoke({'topic': USER_PROMPT})

    with open('current_strategy.json', 'w') as f:
        if hasattr(result, 'model_dump'):
             json.dump(result.model_dump(), f)
        else:
             json.dump(result.dict(), f)
             
    log_file = log_to_daily_jsonl(result, folder="logs")
    print(f"✅ Success! Strategy saved to {log_file}")
    
except Exception as e:
    print(f"❌ Error: {e}")

Generating Strategy JSON...
✅ Success! Strategy saved to logs\2026-02-05_pinterest.jsonl
